In [3]:
# required libraries
import csv
import json
import pandas as pd
import datetime as dt
import sqlite3
from sqlalchemy import create_engine
import geocoder
from itertools import islice
import time

In [4]:
# convert CSV data to DataFrame
csvfile = open('./data/restaurant-and-market-health-violations.csv', 'r')
pdCSV = pd.DataFrame.from_csv(csvfile,index_col=24)
pdCSV.head()
csvfile.close()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
# filter data for the July 2018 only
pdDate = pdCSV.loc[(pdCSV['activity_date'] > '2018-06-30')&(pdCSV['activity_date'] <='2018-07-31')]
pdDate = pdDate.reset_index()
pdDate.head()

,row_id,serial_number,activity_date,facility_name,violation_code,violation_description,violation_status,points,grade,facility_address,...,owner_id,owner_name,pe_description,program_element_pe,program_name,program_status,record_id,score,service_code,service_description
0,DA01KEPWCF045,DA01KEPWC,2018-07-18T00:00:00.000,LA FARFALA COFFE SHOP,F045,# 45. Sleeping quarters,OUT OF COMPLIANCE,1,A,346 N FORD BLVD,...,OW0185212,DANERY S. VELASQUEZ,RESTAURANT (0-30) SEATS MODERATE RISK,1631,LA FARFALA COFFE SHOP,ACTIVE,PR0173373,93,1,ROUTINE INSPECTION
1,DA006DSLSF006,DA006DSLS,2018-07-26T00:00:00.000,GARCONS DE CAFE,F006,# 06. Adequate handwashing facilities supplied...,OUT OF COMPLIANCE,2,A,541 S SPRING ST # 114,...,OW0245826,KC INVESTMENT LLC,RESTAURANT (0-30) SEATS MODERATE RISK,1631,GARCONS DE CAFE,ACTIVE,PR0201189,95,1,ROUTINE INSPECTION
2,DA006DSLSF040,DA006DSLS,2018-07-26T00:00:00.000,GARCONS DE CAFE,F040,"# 40. Plumbing: Plumbing in good repair, prope...",OUT OF COMPLIANCE,1,A,541 S SPRING ST # 114,...,OW0245826,KC INVESTMENT LLC,RESTAURANT (0-30) SEATS MODERATE RISK,1631,GARCONS DE CAFE,ACTIVE,PR0201189,95,1,ROUTINE INSPECTION
3,DA006DSLSF052,DA006DSLS,2018-07-26T00:00:00.000,GARCONS DE CAFE,F052,# 01b. Food safety certification,OUT OF COMPLIANCE,2,A,541 S SPRING ST # 114,...,OW0245826,KC INVESTMENT LLC,RESTAURANT (0-30) SEATS MODERATE RISK,1631,GARCONS DE CAFE,ACTIVE,PR0201189,95,1,ROUTINE INSPECTION
4,DA00GQVTQF008,DA00GQVTQ,2018-07-19T00:00:00.000,ACE SUSHI @ VONS,F008,# 08. Time as a public health control; procedu...,OUT OF COMPLIANCE,2,A,4030 S CENTINELA AVE,...,OW0208935,ASIANA MANAGEMENT GR,RESTAURANT (0-30) SEATS HIGH RISK,1632,ACE SUSHI @ VONS,ACTIVE,PR0188606,96,1,ROUTINE INSPECTION


In [13]:
# save filtered CSV data as JSON file
pdDate.to_csv(r'./data/restaurant-and-market-health-violations_temp.csv')
pdDate.to_json(r'./data/restaurant-and-market-health-violations.json', orient='records')
pdDate.columns

Index(['row_id', 'serial_number', 'activity_date', 'facility_name',
       'violation_code', 'violation_description', 'violation_status', 'points',
       'grade', 'facility_address', 'facility_city', 'facility_id',
       'facility_state', 'facility_zip', 'employee_id', 'owner_id',
       'owner_name', 'pe_description', 'program_element_pe', 'program_name',
       'program_status', 'record_id', 'score', 'service_code',
       'service_description'],
      dtype='object')

In [7]:
# save filtered CSV data as CSV file and Export DataFrame to SQLLITE
pdDate.to_csv(r'./data/restaurant-and-market-health-violations-0718.csv')
engine = create_engine('sqlite:///data/los_angeles.db', echo=False)
pdDate.to_sql('losangeles', con=engine, if_exists='append')

IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: losangeles.row_id [SQL: 'INSERT INTO losangeles (row_id, serial_number, activity_date, facility_name, violation_code, violation_description, violation_status, points, grade, facility_address, facility_city, facility_id, facility_state, facility_zip, employee_id, owner_id, owner_name, pe_description, program_element_pe, program_name, program_status, record_id, score, service_code, service_description) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)'] [parameters: (('DA01KEPWCF045', 'DA01KEPWC', '2018-07-18T00:00:00.000', 'LA FARFALA COFFE SHOP', 'F045', '# 45. Sleeping quarters', 'OUT OF COMPLIANCE', 1, 'A', '346 N FORD BLVD', 'LOS ANGELES', 'FA0179718', 'CA', '90022-1121', 'EE0001049', 'OW0185212', 'DANERY S. VELASQUEZ', 'RESTAURANT (0-30) SEATS MODERATE RISK', 1631, 'LA FARFALA COFFE SHOP', 'ACTIVE', 'PR0173373', 93, 1, 'ROUTINE INSPECTION'), ('DA006DSLSF006', 'DA006DSLS', '2018-07-26

In [ ]:
# generate DataFrame to contain location data
dfLoc = pdDate[['facility_id','facility_address','facility_city','facility_state','facility_zip']].drop_duplicates()
dfLoc
dfLoc.set_index('facility_id')

dfLoc.head()

In [ ]:
# geocode location data
LIMIT = 1800
count = 0
for index, row in  islice(dfLoc.iterrows(), LIMIT):
    addr = row['facility_address'].replace("1/2","").split("#")
    location = addr[0].replace("1/4","")+ ", "+row['facility_city']+ ", "+row['facility_state']+ " "+row['facility_zip']
    print(location)
    g = geocoder.mapbox(location,key= "pk.eyJ1Ijoiam9yZ2VwaXJlcyIsImEiOiJjanZvajBudGMwYmptNDRxbG95cWU0ZW0yIn0.x51Y6jI0l8i17yi1qdANpA")
    print(f'{g.lat} {g.lng}') 
    dfLoc.loc[index,'lat'] = g.lat
    dfLoc.loc[index,'lng'] = g.lng
    if(count == 599):
        time.sleep(20)

In [ ]:
dfLoc.count()

In [ ]:
# post geocode to locations table
dfLoc.to_sql('locations', con=engine, if_exists='append')

In [ ]:
# create scoring dataframe
dfScoring = pdDate[['facility_id','score','grade']]
dfScoring.head()
dfScoring.to_sql('scoring', con=engine, if_exists='append')